# BBox Predictions

## Imports

In [ ]:
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from models.u_net import UNet
import utils
import random

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Preparing

In [ ]:
input_size = 128
train_path = "inputs/train/{}.jpg" 
train_mask_path = "inputs/train_masks/{}_mask.gif"
df_train = pd.read_csv('inputs/train_masks.csv')
ids_train = df_train['img'].map(lambda s: s.split('.')[0])
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

print('Training on {} samples'.format(len(ids_train_split)))
print('Validating on {} samples'.format(len(ids_valid_split)))

GT_bbox_file_path = 'inputs/train_bbox.csv'
GT_bboxes = utils.get_bboxes(GT_bbox_file_path)

pred_bbox_file_path = 'inputs/train_bbox2.csv'
pred_bboxes = utils.get_bboxes(pred_bbox_file_path)

def valid_generator(batch_size):
    return utils.valid_generator(train_path, train_mask_path, ids_valid_split, input_size, batch_size, pred_bboxes)

failed_preds_ids = []
for id in ids_valid_split:
    (gx1,gy1,gx2,gy2) = tuple(GT_bboxes[id])
    (px1,py1,px2,py2) = tuple(pred_bboxes[id])
    if px1-gx1 >= 10 or py1-gy1 >= 10 or gx2-px2 >= 10 or gy2-py2 >= 10:
        failed_preds_ids.append(id)

## Create Model

In [ ]:
#U-Net:
model = UNet((input_size, input_size, 3), filters=64, depth=4, activation=lambda x: PReLU()(x))
model.compile(Adam(), loss='binary_crossentropy')
model.load_weights('weights/unet-2017-09-14-1443.hdf5')

## Validation

In [ ]:
def np_dice_value(y_true, y_pred):
    smooth = 1.
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

### Prediction

In [ ]:
val_imgs, val_masks = next(valid_generator(len(ids_valid_split)))
val_imgs = np.array(val_imgs)
val_masks = np.array(val_masks)
val_pred_masks = model.predict(val_imgs, batch_size=1)

### Histogram

In [ ]:
failed_preds_indices = [i for i, id in enumerate(ids_valid_split.values) if id in failed_preds_ids]
failed_val_masks = [val_masks[i] for i in failed_preds_indices]
failed_val_pred_masks = [val_pred_masks[i] for i in failed_preds_indices]

failed_masks_val_dices = [np_dice_value(mask, pred_mask) for (mask, pred_mask) in zip(failed_val_masks, failed_val_pred_masks)]
hist, bins = np.histogram(failed_masks_val_dices, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

masks_val_dices = [np_dice_value(mask, pred_mask) for (mask, pred_mask) in zip(val_masks, val_pred_masks)]
hist, bins = np.histogram(masks_val_dices, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()


### Visualization

In [ ]:
indices = failed_preds_indices[220:250] #random.sample(failed_preds_indices, 50)
for index in indices:
    id = ids_valid_split.values[index]
    utils.show_mask(train_path.format(id), val_masks[index].squeeze(), val_pred_masks[index].squeeze(),
                    show_img=True, bbox = pred_bboxes[id])